# Deploying an STL Monitoring Service for the PT

In this notebook, we will deploy an STL monitoring service for the PT using `rtamt`.

This service will check whether the controller produced [PT_script-2dof_hybrid_test_bench](../hybrid-test-bench/PT_script-2dof_hybrid_test_bench.ipynb) is working as intended. In particular, it will run periodically, and:
1. Queries the time series database InfluxDB to get a batch of data from running the [PT_emulator](../hybrid-test-bench/PT_emulator.ipynb).
2. Evaluates an STL property on that specification.
3. Pushes the robustness of the specification back into InfluxDB.

In [6]:
# Check pre-requisite scripts are available.

import os

# Get the current working directory. Should be hybrid-test-bench.
current_dir = os.getcwd()

assert os.path.basename(current_dir) == 'hybrid-test-bench', 'Current directory is not hybrid-test-bench'

# Get the parent directory. Should be the root of the repository
parent_dir = current_dir

# Check that the various scripts are available.


# Do we need to create the controller.py script?


path_to_check = os.path.join(parent_dir, 'pt_emulator_service.py')
assert os.path.exists(path_to_check), f'{path_to_check} not found. Run the required notebooks in order.'

path_to_check = os.path.join(parent_dir, 'start_influxdb_rabbitmq.py')
assert os.path.exists(path_to_check), f'{path_to_check} not found. Run the required notebooks in order.'

## Starting the Incubator Hybrid Test Bench

To run the services we will need to open XXXX terminals and run our scripts.

Follow the steps:
1. Open a terminal in [hybrid-test-bench](../hybrid-test-bench/) and run: `python start_influxdb_rabbitmq.py`

2. Open a terminal in [hybrid-test-bench](../hybrid-test-bench/) and run: `python hybrid_test_bench_data_recorder_influx.py`

3. Open a terminal in [hybrid-test-bench](../hybrid-test-bench/) and run: `python pt_emulator_service.py`

(For now we don't have the contoller.py script)

4. At this stage you have the PT working, and data being recorded to the time series DB. Check in the DB management page that this is the case.


**Insert screenshot of a working InfluxDB dashboard**


Now that the system is running, we will deploy the STL monitoring service below, following the same approach.

## Deploying the Monitor

The service below queries a batch of data from the past 1h in InfluxDB, and checks that the temperature drops below the max temperature always within 60 seconds of having exceeded it. This is not always the case, so we expect to see the robustness being negative sometimes.

In [ ]:
# %%writefile stl_monitoring_service.py

# Configure python path to load incubator modules
import sys
import os
import logging
import logging.config
import time
from influxdb_client import InfluxDBClient
from influxdb_client.client.write_api import SYNCHRONOUS
import rtamt

# Get the current working directory. Should be hybrid-test-bench.
current_dir = os.getcwd()

assert os.path.basename(current_dir) == 'hybrid-test-bench', 'Current directory is not hybrid-test-bench'

# Get the parent directory. Should be the root of the repository
parent_dir = current_dir

# The root of the repo should contain the startup folder. Otherwise something went wrong during the inital setup.
assert os.path.exists(os.path.join(parent_dir, 'startup')), 'startup folder not found in the repository root'

# The root of the repo should contain the installation folder. Otherwise something went wrong during the inital setup.
assert os.path.exists(os.path.join(parent_dir, 'installation')), 'installation folder not found in the repository root'

bench_startup_dir = os.path.join(parent_dir, 'startup')

assert os.path.exists(bench_startup_dir), 'hybrid-test-bench startup directory not found'

# Add the parent directory to sys.path
sys.path.append(bench_startup_dir)

from communication.server.rabbitmq import Rabbitmq
from communication.shared.protocol import ROUTING_KEY_STATE, ROUTING_KEY_FORCES
import pt_model as pt_model

class STLMonitoringService:

    # ...

    def start_serving(self):
        self._rabbitmq.start_consuming()

if __name__ == "__main__":
    # Get utility functions to config logging and load configuration
    from software.config import load_config
    from pyhocon import ConfigFactory
    
    # Get logging configuration
    logging.config.fileConfig("logging.conf")

    # Get path to the startup.conf file used in the incubator dt:
    startup_conf = os.path.join(os.path.dirname(os.getcwd()), 'incubator_dt', 'software','startup.conf')
    assert os.path.exists(startup_conf), 'startup.conf file not found'

    # The startup.conf comes from the incubator dt repository.
    config = ConfigFactory.parse_file(startup_conf)
    service = STLMonitoringService(rabbitmq_config=config["rabbitmq"], influxdb_config=config["influxdb"])

    service.setup()
    
    # Start the STLMonitoringService
    service.start_serving()

Running the above script from a terminal, and then opening the InfluxDB page, should give you the following results:

**Insert the two screenshots here once it works - one for data (forces & displacements) and one for robustness**

As can be seen, the robustness goes negative ...

**And explain the robustness...**